In [1]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def treasury_data_retrieval():
    df = pd.read_csv('../data/us_treasury_rates.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', ascending=True, inplace=True)
    df = df.reset_index(drop=True)

    return df

df = treasury_data_retrieval()

n = len(df)
h = 5

df_flat = pd.DataFrame()
for id in range(1, n):
    row = df.iloc[id:(id+h), 1:].stack().reset_index(drop=True).to_frame().T
    df_flat = pd.concat([df_flat, row], ignore_index=True)

for id in range(1, 13):
    df_flat[f'y_{id}'] = df.iloc[:(n-1), id]

df_flat['Date'] = df['Date']
df_flat.dropna(inplace=True)
df_flat.columns = df_flat.columns.astype(str)
df_flat.set_index('Date', inplace=True)

df_flat.tail()

,0,1,2,3,4,5,6,7,8,9,...,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-22,4.74,4.67,4.62,4.46,4.37,4.21,4.21,4.17,4.21,4.27,...,4.63,4.46,4.42,4.37,4.32,4.30,4.35,4.41,4.67,4.60
2024-11-25,4.74,4.67,4.61,4.45,4.37,4.21,4.21,4.17,4.24,4.30,...,4.62,4.46,4.37,4.21,4.21,4.17,4.21,4.27,4.53,4.45
2024-11-26,4.76,4.70,4.60,4.43,4.34,4.19,4.17,4.11,4.17,4.25,...,4.61,4.45,4.37,4.21,4.21,4.17,4.24,4.30,4.56,4.48
2024-11-27,4.76,4.69,4.58,4.42,4.30,4.13,4.10,4.05,4.10,4.18,...,4.60,4.43,4.34,4.19,4.17,4.11,4.17,4.25,4.52,4.44
2024-11-29,4.75,4.63,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,...,4.58,4.42,4.30,4.13,4.10,4.05,4.10,4.18,4.45,4.36


In [3]:
def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse

In [6]:
import numpy as np

# Parameters for the sliding window
test_size = 10

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': []}

# Prepare data
X, y = df_flat.iloc[:, :-12], df_flat.iloc[:, -12:]
n_inputs = X.shape[1]
n_outputs = y.shape[1]

X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

dataset = dict()
dtype = torch.get_default_dtype()
dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
dataset['test_input'] = torch.from_numpy(X_test.iloc[0, :].values.reshape(1, -1)).type(dtype).to(device)
dataset['test_label'] = torch.from_numpy(y_test.iloc[0, :].values.reshape(1, -1)).type(dtype).to(device)

# Initialize the model
model = KAN(width=[n_inputs, 20, n_outputs], grid=3, k=2, seed=42, device=device)

# Train the model and compute metrics
results = model.fit(dataset, opt="Adam", lamb=0.0, lr=0.001, steps=1000, metrics=(train_mse, test_mse))


feature = dataset['test_input']
output_list = list()
for id in range(1, test_size + 1):
    new = model(feature).cpu().detach().numpy().flatten()
    output_list.append(new)

    old = feature.cpu().detach().numpy().flatten()[(n_outputs):]
    feature = torch.from_numpy(np.append(old, new).reshape(1, -1)).type(dtype).to(device)
    
    
# Store the metrics
fold_results['train_mse'].append(results['train_mse'][-1])
fold_results['test_mse'].append(results['test_mse'][-1])

# Calculate average metrics across all windows
avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])

print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {mean_squared_error(output_list, y_test)}")

checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                         | 0/1000 [00:00<?, ?it/s]

| train_loss: 9.02e-02 | test_loss: 4.13e-02 | reg: 0.00e+00 | : 100%|▉| 997/1000 [00:50<00:00, 20.4/usr/local/python/3.12.1/lib/python3.12/site-packages/kan/MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/usr/local/python/3.12.1/lib/python3.12/site-packages/kan/MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
/usr/local/python/3.12.1/lib/python3.12/site-packages/kan/MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by 

saving model version 0.1
Sliding Window Cross-Validation Results
Average Train MSE: 0.008116570313334799
Average Test MSE: 0.0033573600730667954


In [7]:
df_naive = pd.DataFrame([y_train.iloc[-1]] * test_size, columns=y_train.columns)
mean_squared_error(df_naive, y_test)
# df_naive

np.float64(0.007895000000000004)

In [170]:
pd.DataFrame(output_list)

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.701710,4.697441,4.692555,4.434425,4.417237,4.291638,4.213596,4.244041,4.331675,4.411107,4.546750,4.565399
1,4.696810,4.686230,4.691916,4.411322,4.416324,4.286805,4.208780,4.244006,4.337145,4.398597,4.538313,4.572013
2,4.717723,4.706206,4.687421,4.427498,4.426670,4.292756,4.223912,4.256084,4.314641,4.390477,4.561850,4.564540
3,4.735749,4.705325,4.674255,4.429115,4.405700,4.275694,4.211753,4.250232,4.302157,4.380348,4.567161,4.551091
4,4.736442,4.693415,4.660488,4.447486,4.404631,4.268580,4.220031,4.241285,4.283300,4.361287,4.586536,4.546647
5,4.754402,4.708516,4.673429,4.485057,4.381530,4.236036,4.196573,4.213774,4.253439,4.343939,4.597925,4.523838
6,4.747772,4.714408,4.685828,4.486348,4.384358,4.232549,4.191501,4.210462,4.251944,4.343989,4.590913,4.520903
7,4.756601,4.724469,4.700190,4.491140,4.382271,4.219991,4.179382,4.197861,4.248753,4.338986,4.581324,4.520635
8,4.771489,4.731377,4.705759,4.494129,4.374681,4.205554,4.167015,4.185571,4.242038,4.328522,4.576955,4.520045
9,4.784493,4.744085,4.707671,4.497817,4.368873,4.197332,4.156684,4.178993,4.232813,4.322545,4.573390,4.516843


In [147]:
y_test

,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12
Date,,,,,,,,,,,,
2024-11-20,4.68,4.63,4.62,4.44,4.37,4.31,4.26,4.28,4.34,4.41,4.66,4.59
2024-11-21,4.72,4.67,4.63,4.45,4.39,4.34,4.30,4.30,4.36,4.43,4.68,4.61
2024-11-22,4.72,4.67,4.63,4.46,4.42,4.37,4.32,4.30,4.35,4.41,4.67,4.60
2024-11-25,4.74,4.67,4.62,4.46,4.37,4.21,4.21,4.17,4.21,4.27,4.53,4.45
2024-11-26,4.74,4.67,4.61,4.45,4.37,4.21,4.21,4.17,4.24,4.30,4.56,4.48
2024-11-27,4.76,4.70,4.60,4.43,4.34,4.19,4.17,4.11,4.17,4.25,4.52,4.44
2024-11-29,4.76,4.69,4.58,4.42,4.30,4.13,4.10,4.05,4.10,4.18,4.45,4.36
